In [40]:
import numpy as np
from sys import maxsize
import random

*R* : Set of *m* resources represented as (id, cost?) tuples

In [3]:
m = 10

R = []
R = [(i, np.random.randint(0, 100)) for i in range(m)]
print(R)

[(0, 15), (1, 30), (2, 15), (3, 42), (4, 67), (5, 34), (6, 0), (7, 94), (8, 16), (9, 72)]


*N* : Set of *n* agents as (id, balance?, priority?)

In [4]:
n = 2

N = []
N = [(i, np.random.randint(0, 100), np.random.randint(0, 100)) for i in range(n)]
N

[(0, 42, 77), (1, 59, 40)]

Zero Allocations

In [5]:
def init_allocs(resources, agents):
    allocs = []
    for index, i in enumerate(agents):
        r_i = [0 for x in resources]
        allocs.append(r_i)
        
    return allocs

A = init_allocs(R, N)
print(A)

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]


Random allocations

In [6]:
def rand_allocs(resources, agents):
    allocs = init_allocs(resources, agents)
    for index, i in enumerate(resources):
        allocs[np.random.randint(0, 100)//(100//len(agents))][index] = 1
    return allocs

In [7]:
A = rand_allocs(R, N)
print(A)

[[1, 1, 1, 1, 0, 0, 1, 1, 1, 1], [0, 0, 0, 0, 1, 1, 0, 0, 0, 0]]


Random utility values of resources for both users : U

In [8]:
U = []

for index, i in enumerate(N):
    u_i = random.sample(range(0, 100), len(R))
    U.append(u_i)
    
    
print(U)

[[22, 9, 16, 1, 96, 64, 71, 35, 86, 20], [72, 91, 67, 12, 28, 16, 26, 96, 5, 36]]


Random pair-wise complementing utility values : V

In [9]:
V = []

for index, i in enumerate(N):
    v_ijjp1 = random.sample(range(0, 100), len(R)//2)
    V.append(v_ijjp1)
    
    
print(V)

[[20, 61, 48, 68, 52], [62, 28, 89, 61, 31]]


In [10]:
additive_utililty = lambda alloc, utility : sum(np.array(alloc) * np.array(utility))

def complementing_utility(alloc, utility, p_utility) :
    pairwise_alloc = []
    for i in range(0, len(alloc), 2):
        pairwise_alloc.append(alloc[i] & alloc[i+1])
        
#     print(alloc)
#     print(pairwise_alloc)
#     print(utility)
#     print(p_utility)
    return additive_utililty(alloc, utility) + additive_utililty(pairwise_alloc, p_utility)

In [11]:
print(additive_utililty(A[0], U[0]))
print(complementing_utility(A[0], U[0], V[0]))

260
461


In [12]:
def optimal_nash_prod_wcomp(utility, p_utility) :
    n = len(utility)
    if n :
        m = len(utility[0])
        
    optimal_alloc = np.zeros((n, m))
    opt_payoff = 0
    for i in range(2**(m*n)) :
        curr_alloc = np.array([int(x) for x in list(np.binary_repr(i, width=(m*n)))]).reshape(n, m).tolist()
        req_resources = np.zeros(m)
        payoff = 1
        # print(curr_alloc)
        for j in range(len(curr_alloc)) :
            req_resources += curr_alloc[j]
        if not np.array_equal(req_resources, np.ones(m)) :
            continue
        for j in range(len(curr_alloc)) :
            payoff *= complementing_utility(curr_alloc[j], utility[j], p_utility[j])
            
        if opt_payoff < payoff :
            opt_payoff = payoff
            optimal_alloc = curr_alloc
            
    return(optimal_alloc)

In [13]:
onpa = optimal_nash_prod_wcomp(U, V)

<hr>
#### Envy and envy up to one good calculations
<hr>

In [14]:
def envyMap(allocs, utilities, p_utilities, size_agents):
    envy_map = np.zeros((size_agents, size_agents))
    for i in range(len(envy_map)):
        for j in range(len(envy_map[i])):
            envy_map[i][j] = complementing_utility(allocs[i], utilities[i], p_utilities[i]) \
            - complementing_utility(allocs[j], utilities[j], p_utilities[j])
    return envy_map

In [54]:
def envyMap_upto1(allocs, utilities, p_utilities, size_agents, size_res):
    envy_map = np.zeros((size_agents, size_agents))
    for i in range(len(envy_map)) :
        for j in range(len(envy_map[i])):
            envy_map[i][j] = complementing_utility(allocs[i], utilities[i], p_utilities[i]) \
            - complementing_utility(allocs[j], utilities[i], p_utilities[i])
            if envy_map[i][j] < 0 :
                envy_res_drop1_j = np.zeros(size_res)
                max_pos_envy = - maxsize
                for x in range(len(envy_res_drop1_j)) :
                    new_alloc_j = list(allocs[j])
                    new_alloc_j[x] = 0
                    envy_res_drop1_j[x] = complementing_utility(allocs[i], utilities[i], p_utilities[i]) \
                    - complementing_utility(new_alloc_j, utilities[i], p_utilities[i])
                    # print(envy_res_drop1_j)
                    if envy_res_drop1_j[x] >= max_pos_envy:
                        envy_map[i][j] = envy_res_drop1_j[x]
                        max_pos_envy = envy_res_drop1_j[x]
                
    return envy_map

In [38]:
print(U)
print(V)

[[22, 9, 16, 1, 96, 64, 71, 35, 86, 20], [72, 91, 67, 12, 28, 16, 26, 96, 5, 36]]
[[20, 61, 48, 68, 52], [62, 28, 89, 61, 31]]


Envy map represents the envy between two agents. $envymap_{ij}$ is the amount with which i envies j

In [57]:
rand_a = rand_allocs(R, N)
print("Random allocations : ", rand_a)
print(envyMap(rand_a, U, V, n))
print(envyMap_upto1(rand_a, U, V, n, m))

Random allocations :  [[0, 0, 1, 1, 0, 1, 1, 0, 0, 0], [1, 1, 0, 0, 1, 0, 0, 1, 1, 1]]
[[   0. -208.]
 [ 208.    0.]]
[[  0.  11.]
 [272.   0.]]


In [60]:
rand_a = rand_allocs(R, N)
print("Random allocations : ", rand_a)
print(envyMap(rand_a, U, V, n))
print(envyMap_upto1(rand_a, U, V, n, m))

Random allocations :  [[0, 0, 1, 0, 1, 1, 1, 0, 0, 1], [1, 1, 0, 1, 0, 0, 0, 1, 1, 0]]
[[  0. -23.]
 [ 23.   0.]]
[[  0. 142.]
 [ 76.   0.]]


In [61]:
print("Nash product allocations : ", onpa)
print(envyMap(onpa, U, V, n))
print(envyMap_upto1(onpa, U, V, n, m))

Nash product allocations :  [[0, 0, 0, 0, 1, 1, 0, 0, 1, 1], [1, 1, 1, 1, 0, 0, 1, 1, 0, 0]]
[[   0. -149.]
 [ 149.    0.]]
[[  0.  63.]
 [310.   0.]]
